<a href="https://colab.research.google.com/github/pdrobny/Potential_Talents/blob/main/starred_candidates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Potential Talent - Re-rank based on starred candidates






## Setup

In [1]:
!pip install numpy
!pip install scipy seaborn
!pip install torch torchvision torchaudio
!pip install tensorflow
!pip install torchview

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import logging
import random
import requests
import sys
import regex as re
import plotly.express as px
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, make_scorer, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
from sklearn.feature_selection import RFE
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchview import draw_graph
import torch.nn.functional as F
from transformers import pipeline
warnings.filterwarnings('ignore', category=UserWarning)

print(torch.__version__)
tf.__version__

2.6.0+cu124


'2.18.0'

### Data extraction

In [4]:
df = pd.read_csv('talents.csv')
df


,id,title,sentence_bert_cossim
0,1,innovative and driven professional seeking a r...,1.000000
1,431,aspiring data science professional focused on ...,0.769162
2,544,data analyst data scientist business analyst d...,0.768222
3,833,data analyst turning complex data into actiona...,0.745245
4,199,ms in information systems northeastern univers...,0.727268
...,...,...,...
1260,648,research specialist university of rochester di...,0.079923
1261,730,medical biller at brick pediatric group,0.072848
1262,990,ingeniero elctrico,0.067254
1263,296,company owner at armstrong cleans carpets,0.056890


In [5]:
# remove previous evaluation results
df = df.drop('sentence_bert_cossim', axis=1)
df

,id,title
0,1,innovative and driven professional seeking a r...
1,431,aspiring data science professional focused on ...
2,544,data analyst data scientist business analyst d...
3,833,data analyst turning complex data into actiona...
4,199,ms in information systems northeastern univers...
...,...,...
1260,648,research specialist university of rochester di...
1261,730,medical biller at brick pediatric group
1262,990,ingeniero elctrico
1263,296,company owner at armstrong cleans carpets


# Rank candidates based on job title

In [8]:
# request user input for a job title and use sentence bert to find best matches to df['title']

job_title = input("Enter the job title: ")

model = SentenceTransformer('all-mpnet-base-v2')
job_title_embedding = model.encode(job_title)
df['title_embedding'] = df['title'].apply(lambda x: model.encode(x))

df['sentence_bert_cossim'] = df['title_embedding'].apply(lambda x: util.cos_sim(job_title_embedding, x).item())

# Drop the embedding column if you don't need it anymore
df = df.drop('title_embedding', axis=1)

# Sort the DataFrame by cosine similarity in descending order to get best matches
# request user input for a job title and use sentence bert to find best matches to df['title']

job_title = input("Enter the job title: ")

model = SentenceTransformer('all-mpnet-base-v2')
job_title_embedding = model.encode(job_title)
df['title_embedding'] = df['title'].apply(lambda x: model.encode(x))

df['sentence_bert_cossim'] = df['title_embedding'].apply(lambda x: util.cos_sim(job_title_embedding, x).item())

# Drop the embedding column if you don't need it anymore
df = df.drop('title_embedding', axis=1)

# Sort the DataFrame by cosine similarity in descending order to get best matches
df_ranked = df.sort_values(by='sentence_bert_cossim', ascending=False)

print(f"Finding top candidates for: {job_title}")
print(df_ranked[['id','title', 'sentence_bert_cossim']])

Enter the job title: data analyst
Enter the job title: data analyst
Finding top candidates for: data analyst
        id                                              title  \
195   1100                                       data analyst   
194    589                                       data analyst   
200    785                                       data analyst   
201    140                                       data analyst   
207    741                                       data analyst   
...    ...                                                ...   
1246   581  researcher we trilogy physics honor society ma...   
1262   990                                 ingeniero elctrico   
1254   680  full-stack developer react spring boot firebas...   
1259  1201                                       memento mori   
1264   551  python arcpy arcgis pro esri products geoai do...   

      sentence_bert_cossim  
195               1.000000  
194               1.000000  
200               1.000

# Rank Candidates based on starred candidates

In [9]:
# compare to up to 5 ids selected by prompt

starred_ids_input = input("Enter up to 5 starred candidate IDs separated by commas: ")
starred_ids = [int(id.strip()) for id in starred_ids_input.split(',') if id.strip()]

if len(starred_ids) > 5:
    print("Warning: Only considering the first 5 IDs.")
    starred_ids = starred_ids[:5]

# Identify the rows in the dataframe corresponding to the starred IDs
starred_candidates_df = df_ranked[df_ranked['id'].isin(starred_ids)]

if not starred_candidates_df.empty:
    # Calculate the average embedding of the starred candidates' titles
    avg_starred_embedding = np.mean(starred_candidates_df['title'].apply(lambda x: model.encode(x)).tolist(), axis=0)

    # Recalculate cosine similarity using the average starred embedding
    df_ranked['reranked_cossim'] = df_ranked['title'].apply(lambda x: util.cos_sim(avg_starred_embedding, model.encode(x)).item())

    # Sort the DataFrame by the new reranked cosine similarity
    df_reranked = df_ranked.sort_values(by='reranked_cossim', ascending=False)

    print("\nReranked results based on starred candidates:")
    print(df_reranked[['id','title', 'reranked_cossim']])
else:
    print("\nNo starred candidates found with the provided IDs. Displaying original ranking.")
    print(df_ranked[['id','title', 'sentence_bert_cossim']])

Enter up to 5 starred candidate IDs separated by commas: 833

Reranked results based on starred candidates:
        id                                              title  reranked_cossim
3      833  data analyst turning complex data into actiona...         1.000000
12     134  data analyst turning data into actionable insi...         0.879499
10     514  data analyst data science business development...         0.848765
2      544  data analyst data scientist business analyst d...         0.845658
13     542  aspiring data scientist data analyst excel sql...         0.811830
...    ...                                                ...              ...
1246   581  researcher we trilogy physics honor society ma...         0.084555
1258  1111         manager investment risk at cpp investments         0.083317
1262   990                                 ingeniero elctrico         0.072804
1255    91  alumni of california state university channel ...         0.070945
1259  1201             

In [ ]:
df_reranked